In [15]:
import setup 
setup.init_django()

In [16]:
from market.models import StockQuote

In [17]:
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [18]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

6017

In [19]:
total = 0
for obj in qs:
    total += obj.close_price

In [20]:
total / qs.count()

Decimal('410.8609033072959946817350839')

In [21]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 1, 'avg_price': Decimal('249.247879516908')}, {'company': 2, 'avg_price': Decimal('194.198671084337')}, {'company': 4, 'avg_price': Decimal('606.700076438457')}]>

In [22]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by('-time')[:count])
rolling_qs.reverse()

In [23]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 149470 237.0000 237.0000
2 149471 236.9900 236.9950
3 149472 237.0000 236.9966666666666666666666667
4 149473 237.0000 236.9975
5 149474 237.0200 237.0020


In [24]:
qs = StockQuote.objects.filter(company__ticker=ticker, time__range=(start_date, end_date))

In [28]:
frame_start = -(count - 1)
ma_val = qs.annotate(
    ma=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        frame=RowRange(start=frame_start, end=0),
    )
).order_by('-time')

In [29]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

149474 237.0200 237.001999999999 2025-01-11 00:55:00+00:00
149473 237.0000 236.987999999999 2025-01-11 00:50:00+00:00
149472 237.0000 236.979999999999 2025-01-11 00:45:00+00:00
149471 236.9900 236.969999999999 2025-01-11 00:40:00+00:00
149470 237.0000 236.961999999999 2025-01-11 00:30:00+00:00


In [31]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
    ma_5=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        frame=RowRange(start=-4, end=0),
    ),
    ma_20=Window(
        expression=Avg('close_price'),
        order_by=F('time').asc(),
        frame=RowRange(start=-19, end=0),
    )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(149474,
 Decimal('237.0200'),
 Decimal('237.001999999999'),
 Decimal('237.006945000000'))